## Erkennen von verdächtigen Operationen

In [43]:
import pandas as pd
from pandas.tseries.frequencies import to_offset
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Band, ColumnDataSource, HoverTool, RangeTool, CDSView, GroupFilter, Title
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Div, Select
from bokeh.layouts import column
output_notebook()

Loading BokehJS ...

In [2]:
raw_data = pd.read_csv(
    'data\\BOS raw data.csv',
    sep = ';',
    decimal = ',',
    usecols = ['Contract', 'Order Book Limit', 'Snapshot Date', 'Snapshot Time', 'Member','Buy Sell Flag Order Snapshot'],
    parse_dates = {'Timestamp': ['Snapshot Date', 'Snapshot Time']},
    dtype = {'Member' : str},
    index_col = 'Timestamp',
)

In [3]:
contract = 'DEBY Jan 2019'

buysell='Buy'

data = raw_data[(raw_data['Contract'] == contract)&(raw_data["Buy Sell Flag Order Snapshot"]==buysell)].sort_index()

#data.head()

Für diese Zeitreihe werden der gleitende Median und die gleitende Standardabweichung mit einem Fenster (einbezogene zurückliegende Zeitpunkte) von 1 Minute berechnet. Die ersten Einträge fehlen dadurch und werden nachträglich auf den zuerst richtig berechneten Eintrag gesetzt.

In [95]:
# Mittelwert und Standardabweichung aus vorhergehender Minute
data2 = data["Order Book Limit"].rolling(window='1T').agg({"y_median": np.median, "y_std": np.std})

# erste fehlende Werte auf ersten berechneten Wert setzen
data2 = data2.fillna(method='bfill')


In [44]:
#data2.head()

Timedelta('0 days 00:01:00')

### Hampel Filter (MAD)
Nun wird der Bereich berechnet, die alle Werte ohne die Ausreißer enthält. Dazu wird auf den jeweiligen Median die entsprechende Standardabweichung 3 mal addiert bzw. subtrahiert. Alle Werte, die außerhalb dieses Bereiches liegen, werden als verdächtige Operationen deklariert.

In [6]:
df = pd.concat([data, data2], axis=1).reset_index().rename(columns={"Order Book Limit":"OBL"})

df['lower'] = df.y_median - 3*df.y_std
df['upper'] = df.y_median + 3*df.y_std

# Genauigkeit
eps=0.01

outlier=(df["OBL"]>df["upper"]+eps)|(df["OBL"]<df["lower"]-eps)

df["outlier"]=outlier.astype(int)

df_outliers=df[outlier] 

source = ColumnDataSource(df)
source_outliers = ColumnDataSource(df_outliers)

In [53]:
df_count=df.groupby(['Member']).size().reset_index(name='Anzahl_Operationen')
df_count.set_index("Member", inplace=True)
df_count.head()

,Anzahl_Operationen
Member,
111,105
113,57
148,523
153,158
173,2776


### Scatter Plot mit markierten verdächtigen Operationen
In der Grafik wurde ein Scatterplot erstellt. Dabei sind die verdächtigen Operationen rot hervorgehoben. Der Bereich ohne Ausreißer kann gelb hinterlegt werden. Dafür stellt Bokeh die Klasse Band() zur Verfügung. Mit DataTable() ist dann noch die Liste der verdächtigen Operationen angefügt.

In [9]:
TOOLTIPS=[("Time", "@Timestamp{%Y-%m-%d %H:%M:%S}"),("Price", "@OBL{0.00}"), ("Member", "@Member")]
hover_tool=HoverTool(tooltips=TOOLTIPS, formatters={"Timestamp": "datetime"})
TOOLS=["pan","wheel_zoom","box_zoom","reset", "box_select",hover_tool,"xpan"]

In [10]:
quarter_time=df.iloc[round(df.shape[0]/4),0]
middle_time=df.iloc[round(df.shape[0]/2),0]

In [99]:
source_median=ColumnDataSource(df.drop_duplicates(subset=["Timestamp"]))

In [100]:
width=800

p = figure(tools=TOOLS, x_range=(quarter_time, middle_time),x_axis_type="datetime", 
           plot_width=width, plot_height=400, output_backend="webgl")

p.scatter("Timestamp", "OBL", line_color=None, fill_color="lightgray",fill_alpha=1, size=5,source=source)
p.scatter("Timestamp", "OBL", line_color=None, fill_color="red", size=5,legend="verdächtig",source=source_outliers)


band = Band(base='Timestamp', lower='lower', upper='upper', source=source, level='underlay',
            fill_alpha=0.5, line_width=1, line_color='black')
p.add_layout(band)

p.line("Timestamp","y_median", legend="Median", source=source_median)

p.title.text = "BOS ("+buysell+")"
p.xgrid[0].grid_line_color=None
p.xaxis.axis_label = 'Time range'
p.yaxis.axis_label = 'Order Book Limit'

p.toolbar.active_inspect = None

#RangeTool
select = figure(plot_height=100, plot_width=800, y_range=p.y_range, 
                x_range=(df.iloc[0,0]-pd.to_timedelta(to_offset('10T')), df.iloc[-1,0]), 
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.scatter("Timestamp", "OBL", line_color=None, fill_color="red", size=5,source=source_outliers)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

# Data Table mit verdächtigen Operationen
div=Div(text="""verdächtige Operationen""", width=200, height=10)

columns = [
        TableColumn(field="Timestamp", title="Time", formatter=DateFormatter(format="%Y-%m-%d %H:%M:%S")),
        TableColumn(field="Member", title="Member ID"),        
        TableColumn(field="OBL", title="Order Book Limit")
    ]

data_table=DataTable(source=source_outliers, columns=columns, width=800, height=280)

show(column(p,select,div,data_table))

### Balkendiagramm Anzahl/Anteil verdächtiger Operationen pro Member
Nun kann man die Anzahl der verdächtigen Operationen je Member berechnen und in ein neues DataFrame schreiben.

In [28]:
df_count_outliers=df.groupby(['Member'])["outlier"].agg('sum').reset_index(name='Anzahl_Ausreißer')

df_count_outliers.set_index("Member", inplace=True)

df_count_outliers=pd.concat([df_count_outliers, df_count], axis=1, join='inner')

df_count_outliers=df_count_outliers[df_count_outliers["Anzahl_Ausreißer"]>0]

df_count_outliers["Anteil_Ausreißer_in_Prozent"]=\
    np.round(df_count_outliers["Anzahl_Ausreißer"]*100/df_count_outliers["Anzahl_Operationen"],2)


source_count_outliers=ColumnDataSource(df_count_outliers.reset_index())

# Data Table mit Ausreißern
div=Div(text="""Ausreißer nach Member""", width=200, height=10)

columns = [
        TableColumn(field="Member", title="Member ID"),
        TableColumn(field="Anzahl_Operationen", title="Anzahl Operationen"),
        TableColumn(field="Anzahl_Ausreißer", title="Anzahl verdächtige Operationen", width=450),
        TableColumn(field="Anteil_Ausreißer_in_Prozent", title="Anteil verdächtiger Operationen in Prozent", width=600)
    ]

data_table=DataTable(source=source_count_outliers, columns=columns, width=width, height=150)

show(column(div,data_table))

Mit vbar können die Balken dann dargestellt werden. Durch das Setzen von x_range auf die sortierte Member-Liste weiß Bokeh, dass es sich um eine nominal skalierte x-Achse handelt.

In [13]:
p2= figure(title="Anzahl verdächtige Operationen je Member", 
           plot_height=300,plot_width=width, toolbar_location="below",x_range=sorted(df_count.index, key=lambda x:int(x)))

p2.vbar(x="Member", top="Anzahl_Ausreißer",width=0.5,source=source_count_outliers, line_width=0, line_color=None)

p2.xaxis.axis_label = 'Member'
p2.yaxis.axis_label = 'Anzahl verdächtige Operationen'

show(p2)

### Inner Join mit Member Names
Die Namen der auffälligen Members sollen jetzt auch mit ausgegeben werden. Die Member-Namen sind in der Tabelle member_names.txt abgelegt.

In [72]:
member_names = pd.read_csv(
    'data\\member_names.txt',
    sep = ',',
    dtype = {0: str},
)
member_names.set_index("Member", inplace=True)

In [77]:
member_outliers=pd.concat([member_names, df_count_outliers], axis=1, join='inner')
member_outliers.head(member_outliers.shape[0])

,Member Name,Anzahl_Ausreißer,Anzahl_Operationen,Anteil_Ausreißer_in_Prozent
Member,,,,
288,4B,1,371,0.27
332,4D,5,4695,0.11
353,5B,34,9578,0.35
377,5D,4,3891,0.10
48,6D,1,6,16.67
49,7B,36,536,6.72


### Interaktive Anzeige (nach Member, Buy/Sell, Contract)

In [14]:
clean_data = pd.DataFrame(raw_data)
clean_data['Contract'] = clean_data['Contract'].astype(str)
clean_data['Member'] = clean_data['Member'].astype(str)

def graphic1(doc):
    
    source = ColumnDataSource(clean_data)
    
    #separate datasets
    view_buy = CDSView(source=source, filters=[GroupFilter(column_name='Buy Sell Flag Order Snapshot', group='Buy')])
    view_sell = CDSView(source=source, filters=[GroupFilter(column_name='Buy Sell Flag Order Snapshot', group='Sell')])

    #create plot
    f1 = figure(width=width, height=400, x_axis_type="datetime", output_backend="webgl")
    f1.circle('Timestamp', 'Order Book Limit', color='#210089', source=source, legend='Buy', view=view_buy)
    f1.circle('Timestamp', 'Order Book Limit', color='#267d1b', source=source, legend='Sell', view=view_sell)
    
    f1.xaxis.axis_label = 'Time range'
    f1.yaxis.axis_label = 'Order Book Limit'
    
    f1.legend.location = 'top_left'
    f1.legend.click_policy='hide'
    
    #Select Member
    members = ['all'] + clean_data['Member'].sort_values(ascending=True).unique().tolist()
    select_members = Select(title="Member:", value=members[0], options=members, width=width)
    
    #Select Contracts
    contracts = clean_data['Contract'].sort_values(ascending=True).unique().tolist()    
    select_contracts = Select(title='Contract:', value=contracts[0], options=contracts, width=width)
    
    #Select Buy/Sell
    buy_or_sell = ['Both'] + clean_data['Buy Sell Flag Order Snapshot'].sort_values(ascending=True).unique().tolist()    
    select_buy_or_sell = Select(title='Buy/Sell:', value=buy_or_sell[0], options=buy_or_sell, width=width)

    #Callback Members
    def callback(attr, old, new):
        
        sorted_data = clean_data.copy()
        
        sorted_data = sorted_data[
            #Contracts
            (sorted_data['Contract'] == select_contracts.value)
        ]
        
        #Members
        if(select_members.value != 'All'):
            sorted_data = sorted_data[sorted_data['Member'] == select_members.value]
            
        #Buy/Sell
        if(select_buy_or_sell.value != 'Both'):
            sorted_data = sorted_data[sorted_data['Buy Sell Flag Order Snapshot'] == select_buy_or_sell.value]

        source.data = ColumnDataSource(sorted_data).data
        
    select_members.on_change('value', callback)
    select_buy_or_sell.on_change('value', callback)
    select_contracts.on_change('value', callback)
    
    doc.add_root(column(select_members, select_buy_or_sell, select_contracts, f1))
    
show(graphic1, notebook_url="http://localhost:8888") # notebook_url="http://localhost:8888"